# Luentoviikko 3 - Koneoppimisen perusteet

- Mitä on koneoppiminen?
- Mitä ovat piirteet ja miten ne liittyvät koneoppimiseen
- Miten koneoppimista sovelletaan datatieteessä?
- Onko datatiede kokonaisuudessaan koneoppimista?
- Miten koneoppiminen ja tekoäly liittyvät toisiinsa?

# Muistiinpanot

Osallistuin viikon opetukseen katsomalla luentotallenteen sekä demon panoptosta.

Opetetaan algoritmi koneoppimisen perusteiden mukaisesti
- Koneoppimisen 4 tyyppiä: ohjattu, ohjaamaton, (vahvistettu ja -syväoppiminen)

Ohjatun oppimisen työnkulku
- Raakadata (esim. luottokorttipäätökset)
- Label (ennustettava muuttuja, onko luottokortteja myönnetty vai ei), ei aina saatavilla, johdettava raakadatasta manuaalisesti
- Piirteet - joukko muuttujia, jotka ennustavat tulemaa (ikä, luottotiedto yms.)
- Malli muodostaa tulokset matriisista, opetetaan, luokitteluongelama, false negatives/positives, kumpi virhe kalliimpi (kortti myönnetään vai ei)
- Tulosten ennustaminen mallin perusteella, uusia piirteitä voidaan muodostaa koneoppimisen avulla

Laadukas data -> algoritmien opetus

Piirteet
- muuttujilla oltava merkitystä, olennaiset piirteet erotettava datasta
- esim. "Punavuori" piirre ei ole hyödyllinen Berliinissä, jokin muu piirre ennustaa hintaa paremmin
- esim. piirteet jotka ennustavat hintaa (käyttis, näyttö, muisti...)
- piirteiden valinta riippuu mitä halutaan ennustaa (hinta, laitekehitys, palauteen laatu)
- ohjaamaton oppiminen, palautteiden klusterointi / ryhmittely
- laiteet jotka keräävät dataa 24/7, esim. verkkokamerat/sateliitit tallentavat vain piirteet tilan säästämiseksi, kaikkea ei haluta tallentaa